## 🎯 Desenvolvimento do Agente DQN para Alocação de Ativos

Este notebook implementa a Etapa 6 do projeto de Reinforcement Learning, dedicada à construção prática de um agente baseado no algoritmo Deep Q-Network (DQN), aplicado à alocação dinâmica de uma carteira com três ativos: VALE3, PETR4 e BRFS3.

O agente será treinado em um ambiente simulado (`PortfolioEnv`), configurado com base nos dados históricos processados previamente. A arquitetura será baseada em PyTorch com suporte a CUDA, validando os aprendizados teóricos sobre a função Q, replay buffer e políticas $\epsilon$-greedy.

Etapas implementadas neste notebook:

1. Verificação do ambiente e suporte à GPU;
2. Definição da rede neural DQN;
3. Construção do ambiente simulado `PortfolioEnv`;
4. Configuração do buffer de experiência e parâmetros de treinamento;
5. Execução do treinamento inicial do agente;
6. Análise dos resultados preliminares.


In [74]:
# Bibliotecas principais para RL e Data Science
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque
import matplotlib.pyplot as plt
import seaborn as sns

# Verifica se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"✅ Dispositivo ativo: {device}")
print(f"CUDA disponível? {torch.cuda.is_available()}")


✅ Dispositivo ativo: cuda
CUDA disponível? True


## 🧠 Arquitetura da Rede Neural DQN

A rede neural utilizada neste projeto segue uma arquitetura simples e eficiente, adequada ao tamanho do vetor de estado calculado com base em indicadores técnicos e variáveis da carteira.

A rede recebe como entrada o vetor de estado do ambiente simulado (`PortfolioEnv`) e retorna os valores Q estimados para cada ação possível. A estrutura adotada é composta por:

- Camada de entrada: dimensão igual ao tamanho total do vetor de estado;
- Camada oculta 1: 128 neurônios com ativação ReLU;
- Camada oculta 2: 64 neurônios com ativação ReLU;
- Camada de saída: 9 neurônios, correspondentes às ações discretas possíveis (comprar, vender ou manter) para cada um dos três ativos.

A saída da rede representa o valor esperado de recompensa (Q-value) para cada ação, dada a configuração atual do ambiente.


In [75]:
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.net(x)


## 🌍 Ambiente Simulado: `PortfolioEnv`

O ambiente `PortfolioEnv` simula o comportamento de um mercado financeiro com três ativos (VALE3, PETR4, BRFS3), considerando variáveis de estado, posições mantidas pelo agente e saldo de caixa disponível.

A interface segue o padrão `gym.Env`, com os principais métodos:

- `reset()`: reinicializa o ambiente para o primeiro dia útil, zera posições e caixa, e retorna o primeiro estado observável;
- `step(action_vector)`: executa as ações sobre os ativos, atualiza posições, saldo e calcula a recompensa;
- `render()`: imprime informações do estado atual da carteira — útil para fins de depuração e visualização em tempo real durante o treinamento;
- Atributos internos controlam a linha temporal, recompensas acumuladas e histórico da carteira.

As ações são vetores discretos com tamanho igual ao número de ativos. Para cada ativo, a ação pode ser:

- `0`: manter posição;
- `1`: comprar uma unidade (caso haja saldo);
- `2`: vender uma unidade (caso haja posição).

A recompensa é definida com base na variação do valor total da carteira entre `t` e `t+1`, incluindo caixa e valor de mercado das posições. Custos de transação podem ser incorporados como penalidade.


In [76]:
import gym
from gym import spaces
import numpy as np

class PortfolioEnv(gym.Env):
    def __init__(self, df, initial_cash=1.0):
        super(PortfolioEnv, self).__init__()

        self.df = df.reset_index(drop=True)
        self.n_assets = 3
        self.initial_cash = initial_cash

        # 🚨 Novo espaço de ações: 3^3 = 27 combinações
        self.action_space = spaces.Discrete(27)

        self.current_step = 0
        self.cash = self.initial_cash
        self.positions = [0] * self.n_assets
        self.history = []

    def reset(self):
        self.current_step = 0
        self.cash = self.initial_cash
        self.positions = [0] * self.n_assets
        self.history.clear()
        return self._get_state()

    def step(self, action):
        # Decodifica a ação combinada: inteiro [0-26] → vetor [a1, a2, a3]
        decoded_actions = np.unravel_index(action, (3, 3, 3))

        # Preços atuais dos ativos
        current_prices = self.df.iloc[self.current_step].values

        # Valor antes das ações
        portfolio_value_before = self.cash + sum([
            self.positions[i] * current_prices[i]
            for i in range(self.n_assets)
        ])

        # Aplica ações
        transaction_cost = 0.001
        for i in range(self.n_assets):
            price = current_prices[i]
            if decoded_actions[i] == 1:  # Comprar
                total_cost = price * (1 + transaction_cost)
                if self.cash >= total_cost:
                    self.positions[i] += 1
                    self.cash -= total_cost
            elif decoded_actions[i] == 2:  # Vender
                if self.positions[i] > 0:
                    total_gain = price * (1 - transaction_cost)
                    self.positions[i] -= 1
                    self.cash += total_gain

        # Avança o tempo
        self.current_step += 1
        done = self.current_step >= len(self.df) - 1
        next_state = self._get_state()

        # Valor depois das ações
        next_prices = self.df.iloc[self.current_step].values
        portfolio_value_after = self.cash + sum([
            self.positions[i] * next_prices[i]
            for i in range(self.n_assets)
        ])

        reward = portfolio_value_after - portfolio_value_before
        info = {
            "valor_portfolio": portfolio_value_after,
            "lucro": reward
        }

        return next_state, reward, done, info

    def _get_state(self):
        market_data = self.df.iloc[self.current_step].values
        return np.concatenate([market_data, self.positions, [self.cash]])

    def render(self, mode='human'):
        print(f"Step: {self.current_step} | Caixa: {self.cash:.2f} | Posições: {self.positions}")


## 🔁 Lógica de Transição do Ambiente: Implementação do Método `step()`

O método `step()` é o coração do ambiente `PortfolioEnv`. Ele define como o agente interage com o mercado e como essa interação afeta o estado da carteira. A implementação segue estas etapas:

1. **Registro do valor da carteira antes das ações**  
   Calcula o valor da carteira no tempo `t`, somando o caixa disponível e o valor de mercado das posições atuais com os preços do dia corrente.

2. **Execução das ações de compra, venda ou manutenção**  
   Para cada ativo:
   - `1` (compra): se houver saldo suficiente, reduz o caixa e incrementa a posição;
   - `2` (venda): se houver posição, reduz a quantidade do ativo e adiciona o valor ao caixa;
   - `0` (manter): nenhuma alteração.

   Cada operação de compra ou venda acarreta um **custo de transação fixo**, deduzido do caixa disponível. Isso reflete taxas de corretagem, spread e custos operacionais — incentivando o agente a evitar operações excessivas.

3. **Cálculo da recompensa**  
   Após as ações, avança para o próximo dia (`t+1`) e calcula o novo valor da carteira.  
   A recompensa é definida como a **variação percentual do valor da carteira** entre `t` e `t+1`.  
   Se o episódio estiver no último dia útil (`t+1` não existir), a recompensa é zero.

4. **Atualização do passo temporal e retorno dos resultados**  
   - `next_state`: novo vetor de observação;
   - `reward`: feedback para o agente;
   - `done`: sinaliza fim do episódio;
   - `info`: dicionário com dados complementares como o valor atual da carteira.

Este método transforma a simulação em um ambiente dinâmico realista, no qual o agente precisa aprender a alocar recursos de forma estratégica e econômica, ponderando retorno e custo de transação.


In [77]:
def step(self, action):
    """
    Aplica o vetor de ações fornecido (um por ativo), atualiza as posições e o caixa,
    avança o tempo e calcula a recompensa como a variação percentual da carteira líquida.
    """

    # Definição do custo fixo por transação (compra ou venda)
    transaction_cost = 0.001  # 0.1% do valor da operação

    # Preços atuais dos ativos (tempo t)
    current_prices = self.df.iloc[self.current_step].values

    # Valor da carteira antes das ações
    portfolio_value_before = self.cash + sum([
        self.positions[i] * current_prices[i]
        for i in range(self.n_assets)
    ])

    # Aplicação das ações
    for i in range(self.n_assets):
        price = current_prices[i]
        if action[i] == 1:  # Comprar
            total_cost = price * (1 + transaction_cost)
            if self.cash >= total_cost:
                self.positions[i] += 1
                self.cash -= total_cost


## ✅ Finalização do Ambiente Simulado `PortfolioEnv`

Com a implementação completa do método `step()`, o ambiente `PortfolioEnv` está agora totalmente funcional e pronto para ser utilizado no treinamento do agente de Reinforcement Learning.

Esse ambiente representa, com realismo e controle, um cenário simplificado de mercado no qual o agente:

- Observa indicadores técnicos e seu próprio portfólio (estado);
- Decide diariamente se deve comprar, vender ou manter posição (ação);
- Recebe uma recompensa baseada na evolução da carteira (retorno financeiro ajustado por custo de transação).

A inclusão de **custos fixos por operação** torna o desafio mais alinhado com o mundo real, penalizando estratégias com excesso de transações e incentivando decisões eficientes.

Além disso, o ambiente já é compatível com algoritmos baseados em `gym`, podendo ser diretamente integrado ao loop de treinamento do DQN, Replay Buffer e política de exploração ε-greedy.

A partir deste ponto, seguimos para a configuração da estrutura de aprendizado propriamente dita, que inclui:

1. Buffer de experiências (Replay Buffer);
2. Política de exploração baseada em ε-greedy;
3. Loop de treinamento com atualização da rede Q e rede alvo.

Com isso, encerramos a Etapa 6.3 — *Definição do Ambiente de Simulação*.


## 🧱 Replay Buffer: Armazenamento de Experiências para Aprendizado

O **Replay Buffer** (ou memória de experiência) é um componente essencial do algoritmo DQN. Ele armazena tuplas de experiência no formato:

(state, action, reward, next_state, done)


Essas experiências representam interações passadas do agente com o ambiente e são usadas para treinar a rede DQN de forma mais estável e eficiente.

### 🎯 Objetivos do Replay Buffer

- **Descorrelacionar as amostras**: ao usar amostras aleatórias em vez de sequenciais, evitamos instabilidades que ocorrem por dados altamente correlacionados.
- **Reutilizar experiências antigas**: mesmo ações passadas que foram ruins (ou boas) podem ser úteis para o aprendizado atual.
- **Suporte ao aprendizado offline**: a rede aprende a partir do histórico, não apenas da última interação.

### ⚙️ Estrutura

O buffer será implementado como uma estrutura de dados circular (FIFO), com capacidade máxima definida (ex: 100.000 interações). Quando o limite é atingido, os dados mais antigos são descartados automaticamente.

A classe `ReplayBuffer` terá os seguintes métodos principais:

- `add(state, action, reward, next_state, done)`: armazena uma nova experiência;
- `sample(batch_size)`: retorna um conjunto aleatório de experiências para treino;
- `__len__()`: retorna o número atual de elementos armazenados.

Essa abordagem permite treinar a rede em mini-batches a partir de interações variadas, melhorando a generalização da política aprendida.



In [78]:
import random
from collections import deque
import numpy as np
import torch

class ReplayBuffer:
    def __init__(self, capacity, device="cpu"):
        """
        Inicializa o buffer de experiência.
        - capacity: número máximo de experiências armazenadas.
        - device: 'cpu' ou 'cuda' para onde os tensores serão enviados.
        """
        self.capacity = capacity
        self.memory = deque(maxlen=capacity)  # estrutura circular FIFO
        self.device = device

    def add(self, state, action, reward, next_state, done):
        """
        Armazena uma nova experiência no buffer.
        """
        self.memory.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        """
        Retorna um mini-batch aleatório de experiências do buffer.
        Os dados são convertidos para tensores PyTorch com tipos explícitos.
        """
        batch = random.sample(self.memory, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        # Conversão segura para arrays numéricos
        states = torch.tensor(np.stack([np.array(s, dtype=np.float32) for s in states]), dtype=torch.float32).to(self.device)
        next_states = torch.tensor(np.stack([np.array(s, dtype=np.float32) for s in next_states]), dtype=torch.float32).to(self.device)

        actions = torch.tensor(actions, dtype=torch.int64).to(self.device)
        rewards = torch.tensor(rewards, dtype=torch.float32).to(self.device)
        dones = torch.tensor(dones, dtype=torch.float32).to(self.device)

        return states, actions, rewards, next_states, dones

    def __len__(self):
        """
        Retorna o número atual de experiências armazenadas.
        """
        return len(self.memory)


## 🎲 Política de Exploração ε-Greedy (com Explotação)

Durante o treinamento do agente de Reinforcement Learning, é importante encontrar um equilíbrio entre:

- **Exploração** (explore): testar ações novas que podem levar a descobertas inesperadas;
- **Explotação** (exploit): executar ações que já parecem gerar boas recompensas, com base na política aprendida até o momento.

A estratégia **ε-greedy** regula esse equilíbrio com uma regra simples:

1. Em cada passo de decisão, o agente **gera um número aleatório entre 0 e 1**;
2. Se esse número for **menor que ε (epsilon)**, o agente escolhe uma **ação aleatória** (exploração);
3. Caso contrário, ele escolhe a **melhor ação segundo a rede Q atual** (explotação da política aprendida).

### 🔄 Decaimento de ε

Para permitir aprendizado eficiente:

- Iniciamos com `ε = 1.0` (exploração máxima);
- Reduzimos gradualmente até um valor mínimo como `ε = 0.05`;
- Isso garante **ampla exploração no início** e **explotação estável ao final**.

Essa política evita que o agente se prenda prematuramente a ações subótimas e promove um aprendizado adaptativo ao ambiente.

Implementaremos uma função `select_action()` que aplica a regra ε-greedy com suporte a CUDA, retornando vetores de ação compatíveis com o ambiente `PortfolioEnv`.


In [79]:
import numpy as np
import torch

def select_action(state, q_net, epsilon):
    """
    Estratégia ε-greedy para seleção de ação combinada (0 a 26).
    """
    if np.random.rand() < epsilon:
        # Exploração: sorteia uma ação aleatória
        return np.random.randint(27)
    else:
        # Explotação: escolhe a melhor ação via rede Q
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(q_net.device)
        with torch.no_grad():
            q_values = q_net(state_tensor)
        return torch.argmax(q_values).item()


## 🌐 Instanciação do Ambiente de Simulação

Nesta etapa, carregamos o vetor de estado final `vetor_portfolio.csv`, que contém os dados combinados dos três ativos (VALE3, PETR4, BRFS3) junto com as variáveis internas do agente.  
Esse vetor representa o ambiente observado pelo agente a cada passo, e será usado para instanciar o `PortfolioEnv`, que simula a dinâmica do mercado e da carteira de investimentos.


In [80]:
import pandas as pd

# Caminho do vetor final de estados
caminho_dados = "../dados/vetor_portfolio.csv"
df = pd.read_csv(caminho_dados)

# Remoção da coluna de datas (caso exista)
if "Date" in df.columns:
    df = df.drop(columns=["Date"])
    print("🧹 Coluna 'Date' removida do vetor de estado.")

# Instanciação do ambiente com o vetor corrigido
env = PortfolioEnv(df)

# Teste de saída
estado_inicial = env.reset()
print("✅ Ambiente criado com sucesso.")
print("📐 Dimensão do vetor de estado:", estado_inicial.shape)


🧹 Coluna 'Date' removida do vetor de estado.
✅ Ambiente criado com sucesso.
📐 Dimensão do vetor de estado: (8,)


## 🧠 Inicialização das Redes DQN e Componentes de Treinamento

Nesta etapa, criamos a rede principal `q_net` e a rede alvo `q_target` com base na dimensão do vetor de estado obtido do ambiente.  
Também instanciamos o otimizador `Adam`, a função de perda `MSELoss` e o `ReplayBuffer`, que será usado para amostragem de transições durante o treinamento do agente.


In [81]:
# 🔢 Hiperparâmetros do modelo
learning_rate = 1e-4
gamma = 0.99
batch_size = 64
buffer_capacity = 100_000
target_update_freq = 500

# 🎲 Exploração
epsilon = 1.0
epsilon_min = 0.05
epsilon_decay = 0.995

max_episodes = 300
min_buffer_size = 1000

# ⚙️ Inicialização do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 🧠 Instanciação das redes DQN
input_dim = estado_inicial.shape[0]
output_dim = 9  # 3 ativos × 3 ações possíveis: [manter, comprar, vender]

q_net = DQN(input_dim=input_dim, output_dim=output_dim).to(device)
q_target = DQN(input_dim=input_dim, output_dim=output_dim).to(device)
q_target.load_state_dict(q_net.state_dict())  # Inicializa a rede alvo com os mesmos pesos

# 🧰 Otimizador e função de perda
optimizer = optim.Adam(q_net.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

# 🎒 Replay Buffer
buffer = ReplayBuffer(capacity=buffer_capacity, device=device)

print("✅ Redes e componentes de treinamento inicializados com sucesso")


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## 🔁 Loop de Treinamento do Agente DQN

Esta célula executa o treinamento do agente ao longo de vários episódios.  
A cada episódio, o agente interage com o ambiente, armazena experiências no `ReplayBuffer`, e atualiza sua rede DQN com base em amostras aleatórias dessas experiências.  
As métricas de desempenho são monitoradas continuamente, e o treinamento é interrompido se um critério de estabilidade for atingido (Sharpe Ratio, Hit Ratio e lucro positivo estáveis).


In [ ]:
# 📈 Inicialização de buffers de métricas
from collections import deque
import numpy as np

reward_history = []
sharpe_window = deque(maxlen=10)
hit_window = deque(maxlen=10)

step_count = 0

for episode in range(max_episodes):
    state = env.reset()
    done = False
    total_reward = 0
    rewards = []

    while not done:
        action = select_action(state, q_net, epsilon)  # função definida previamente
        next_state, reward, done, info = env.step(action)
        buffer.add(state, action, reward, next_state, done)

        state = next_state
        total_reward += reward
        rewards.append(reward)
        step_count += 1

        # 🎯 Treinamento por mini-batch (apenas após preenchimento mínimo do buffer)
        if len(buffer) > min_buffer_size:
            states, actions, rewards_b, next_states, dones = buffer.sample(batch_size)

            # Predição atual
            q_values = q_net(states)
            q_pred = q_values.gather(1, actions.view(-1, 1)).squeeze()

            # Valor alvo com rede alvo congelada
            with torch.no_grad():
                q_next = q_target(next_states).max(1)[0]
                q_target_vals = rewards_b + (1 - dones) * gamma * q_next

            # Otimização
            loss = loss_fn(q_pred, q_target_vals)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Atualiza rede alvo
            if step_count % target_update_freq == 0:
                q_target.load_state_dict(q_net.state_dict())

    # 🔄 Decaimento do epsilon
    epsilon = max(epsilon * epsilon_decay, epsilon_min)

    # 📊 Métricas
    reward_history.append(total_reward)
    sharpe_ratio = np.mean(rewards) / (np.std(rewards) + 1e-6)
    hit_ratio = np.mean([r > 0 for r in rewards])

    sharpe_window.append(sharpe_ratio)
    hit_window.append(hit_ratio)

    print(f"Ep {episode+1} | Recompensa: {total_reward:.3f} | Sharpe: {sharpe_ratio:.2f} | Hit: {hit_ratio:.2%} | Eps: {epsilon:.3f}")

    # 🏁 Critério de parada com estabilidade
    if len(sharpe_window) == 10:
        sharpe_ok = np.mean(sharpe_window) > 1.0 and np.std(sharpe_window) < 0.3
        hit_ok = np.mean(hit_window) > 0.55 and np.std(hit_window) < 0.1
        lucro_ok = np.mean(reward_history[-10:]) > 0
        if sharpe_ok and hit_ok and lucro_ok:
            print("🏁 Critério de estabilidade atingido — encerrando o treinamento.")
            break


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
